In [1]:
#!pip install PyPDF2

     |████████████████████████████████| 81kB 1.8MB/s eta 0:00:011
  Stored in directory: /Users/kai/Library/Caches/pip/wheels/53/84/19/35bc977c8bf5f0c23a8a011aa958acd4da4bbd7a229315c1b7
Successfully built PyPDF2


In [13]:
!pip install pdfminer.six

     |████████████████████████████████| 5.6MB 1.9MB/s eta 0:00:01
     |████████████████████████████████| 10.1MB 4.8MB/s eta 0:00:01


In [17]:
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfpage import PDFPage
from io import StringIO

def convert_pdf_to_txt(path):
    rsrcmgr = PDFResourceManager()
    retstr = StringIO()
    codec = 'utf-8'
    laparams = LAParams()
    device = TextConverter(rsrcmgr, retstr, codec=codec, laparams=laparams)
    fp = open(path, 'rb')
    interpreter = PDFPageInterpreter(rsrcmgr, device)
    password = ""
    maxpages = 0
    caching = True
    pagenos=set()

    for page in PDFPage.get_pages(fp, pagenos, maxpages=maxpages, password=password,caching=caching, check_extractable=True):
        interpreter.process_page(page)

    text = retstr.getvalue()

    fp.close()
    device.close()
    retstr.close()
    return text

Banana = convert_pdf_to_txt("../pdfs/0A9Hhx9f3hLSXUHwtrQq7eVJR9AnkcuB.pdf")
print(Banana)

class: 0
text: Gur unaqyr qrgnpurq sebz gur obk.

1




In [20]:
import os
file_list = []
for file in os.listdir("../pdfs/"):
    if file.endswith(".pdf"):
        file_list.append(os.path.join("../pdfs/", file))

In [30]:
pdf_text = convert_pdf_to_txt("../pdfs/0A9Hhx9f3hLSXUHwtrQq7eVJR9AnkcuB.pdf")
print(pdf_text)

print(pdf_text.splitlines())

class: 0
text: Gur unaqyr qrgnpurq sebz gur obk.

1


['class: 0', 'text: Gur unaqyr qrgnpurq sebz gur obk.', '', '1', '', '']


In [41]:
def get_class_text(pdf_text):
    class_label = -1
    text = ''
    str_list = pdf_text.splitlines()
    for str_el in str_list:
        if 'class' in str_el:
            class_label = int(str_el.split(':')[1].strip())
        elif 'text' in str_el:
            text = str_el.split(':')[1].strip()
    return class_label, text
            

In [42]:
class_label, text = get_class_text(pdf_text)
print(pdf_text)

class: 0
text: Gur unaqyr qrgnpurq sebz gur obk.

1




In [43]:
print(class_label)
print(text)

0
Gur unaqyr qrgnpurq sebz gur obk.
